In [1]:
# Cell 1: Imports and helper function definitions

import requests
import json

# Base URL for the running Flask app
BASE_URL = "http://localhost:5000"

def create_character(
    name: str,
    llm_model: str,
    llm_config: dict,
    voice_cloning_settings: dict,
    stt_settings: dict,
    character_image_path: str = None,
    knowledge_base_file_path: str = None,
    voice_cloning_audio_path: str = None,
    style_tuning_file_path: str = None,
) -> dict:
    """
    Calls /create-character with form data + optional files.
    Returns the parsed JSON response.
    """
    url = f"{BASE_URL}/create-character"
    files = {}
    data = {
        "name": name,
        "llm_model": llm_model,
        "llm_config": json.dumps(llm_config),
        "voice_cloning_settings": json.dumps(voice_cloning_settings),
        "stt_settings": json.dumps(stt_settings),
    }

    if character_image_path:
        files["character_image"] = open(character_image_path, "rb")
    if knowledge_base_file_path:
        files["knowledge_base_file"] = open(knowledge_base_file_path, "rb")
    if voice_cloning_audio_path:
        files["voice_cloning_audio"] = open(voice_cloning_audio_path, "rb")
    if style_tuning_file_path:
        files["style_tuning_file"] = open(style_tuning_file_path, "rb")

    resp = requests.post(url, data=data, files=files)
    for f in files.values():
        f.close()
    resp.raise_for_status()
    return resp.json()


def get_characters() -> dict:
    """
    Calls /get-characters to fetch all characters.
    Returns the parsed JSON response.
    """
    url = f"{BASE_URL}/get-characters"
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()


def load_character(character_id: int) -> dict:
    """
    Calls /load-character with JSON {"character_id": ...}.
    Returns the parsed JSON response.
    """
    url = f"{BASE_URL}/load-character"
    payload = {"character_id": character_id}
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    return resp.json()


def ask_question_text(character_id: int, question: str) -> dict:
    """
    Calls /ask-question-text with JSON {"character_id": ..., "question": ...}.
    Returns the parsed JSON response.
    """
    url = f"{BASE_URL}/ask-question-text"
    payload = {"character_id": character_id, "question": question}
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    return resp.json()


def ask_question_audio(character_id: int, audio_file_path: str) -> dict:
    """
    Calls /ask-question-audio, sending character_id and a WAV file.
    Returns the parsed JSON response.
    """
    url = f"{BASE_URL}/ask-question-audio"
    files = {"audio_file": open(audio_file_path, "rb")}
    data = {"character_id": str(character_id)}
    resp = requests.post(url, data=data, files=files)
    files["audio_file"].close()
    resp.raise_for_status()
    return resp.json()

def ask_question_audio_live(character_id: int) -> dict:
    """
    Calls /ask-question-audio-live, sending character_id as JSON.
    Returns the parsed JSON response.
    """
    url = f"{BASE_URL}/ask-question-audio-live"
    payload = {"character_id": character_id}  # Don't convert to string
    resp = requests.post(url, json=payload)   # Use json= instead of data=
    resp.raise_for_status()
    return resp.json()

In [74]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [75]:
import requests
import json

# Base URL for the running Flask app
BASE_URL = "http://localhost:5000"

# Download the Gemma GGUF model using the API endpoint
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    # Make POST request to the download-model endpoint
    response = requests.post(
        f"{BASE_URL}/download-model",
        json={
            "model_name": model_name,
            "model_type": model_type
        },
        timeout=300  # 5 minute timeout for large model downloads
    )
    
    # Check if request was successful
    response.raise_for_status()
    
    # Parse the JSON response
    result = response.json()
    
    if result.get("status") == "success":
        local_path = result.get("model_path")
        print(f"✅ Model downloaded successfully to: {local_path}")
    else:
        print(f"❌ Download failed: {result.get('error', 'Unknown error')}")
        
except requests.exceptions.Timeout:
    print("❌ Download failed: Request timed out (model download may still be in progress)")
except requests.exceptions.RequestException as e:
    print(f"❌ Request failed: {e}")
except Exception as e:
    print(f"❌ Download failed: {e}")

✅ Model downloaded successfully to: ./models/google_gemma-3-4b-it-qat-q4_0-gguf_gemma-3-4b-it-q4_0.gguf


In [ ]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [ ]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_path": "outputs/generated_audio_1748987406.wav",
  "character_name": "J. Robert Oppenheimer 2",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"chunk_id\": 6,\n  \"keywords\": \"mexico nuclear, weapons, project oppenheimer, nuclear, project\",\n  \"source\": \"oppenheimer_knowledge.txt\"\n}\n\nDocument 2:\nHe is often referred to as the \"father of the atomic bomb\" for his role as the scientific director of the Manhattan Project during World War II.\n\nMetadata:\n{\n  \"chunk_id\": 2,\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"keywords\": \"atomic bomb, bomb role, father atomic, role, referred\",\n  \"entities\": \"the Manhattan Project (ORG), World 

In [90]:
# Cell 2: Define configuration for a new character
# Replace placeholders (YOUR_OPENAI_API_KEY, file paths, etc.) as needed.

# LLM configuration (example for OpenAI GPT-3.5)
llm_config = {
    "api_key": "",
    "base_url": "https://api.openai.com/v1",
    "system_prompt": "You are J. Robert Oppenheimer answering questions in his style. Output at MOST 20 words.",
    "max_tokens": 150,
    "temperature": 0.7
}

# llm_config = {
#     "system_prompt": "You are J. Robert Oppenheimer answering questions in his style. Output at MOST 5 words.",
#     "n_ctx": 2048,  # Context window size
#     "n_gpu_layers": 0,  # Number of layers to offload to GPU (0 for CPU-only)
#     "verbose": False
# }

# Voice cloning (TTS) settings
# voice_cloning_settings = {
#     "model": "zonos",
#     "cache_dir": "voice_models_cache",
#     "preprocess_audio": True,
#     "reference_text": " Of course, the initial discovery and its interpretation in early 1939 attracted everybody's interest.",
#     # add any additional TTS parameters here
# }

voice_cloning_settings = {
  "model": "zonos",
  "reference_text": "Reference audio transcription",
  "preprocess_audio": True,
  "language": "en-us"
}

# Speech-to-text (STT) settings
stt_settings = {
    "model": "whisper",
    "model_size": "base"
    # add any additional STT parameters here
}

# File paths (place actual files on disk before running Cell 3)
character_image_path = "./test_data/oppenheimer.png"
knowledge_base_file_path = "./test_data/oppenheimer_knowledge.txt"
voice_cloning_audio_path = "./test_data/oppenheimer_voice.wav"
style_tuning_file_path = "./test_data/oppenheimer_qa.json"


In [ ]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_path": "outputs/generated_audio_1748987406.wav",
  "character_name": "J. Robert Oppenheimer 2",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"chunk_id\": 6,\n  \"keywords\": \"mexico nuclear, weapons, project oppenheimer, nuclear, project\",\n  \"source\": \"oppenheimer_knowledge.txt\"\n}\n\nDocument 2:\nHe is often referred to as the \"father of the atomic bomb\" for his role as the scientific director of the Manhattan Project during World War II.\n\nMetadata:\n{\n  \"chunk_id\": 2,\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"keywords\": \"atomic bomb, bomb role, father atomic, role, referred\",\n  \"entities\": \"the Manhattan Project (ORG), World 

In [91]:
# Cell 3: Create a new character
# This will return a JSON with "character_id", which we’ll capture.
create_resp = create_character(
    name="J. Robert Oppenheimer 10",
    llm_model="gpt-3.5-turbo",
    llm_config=llm_config,
    voice_cloning_settings=voice_cloning_settings,
    stt_settings=stt_settings,
    character_image_path=character_image_path,
    knowledge_base_file_path=knowledge_base_file_path,
    voice_cloning_audio_path=voice_cloning_audio_path,
    style_tuning_file_path=style_tuning_file_path
)

print("Create-character response:")
print(json.dumps(create_resp, indent=2))

# Extract character_id for subsequent calls
character_id = create_resp.get("character_id")
print(f"\nNew character_id = {character_id}")

Create-character response:
{
  "character_id": 14,
  "message": "Character created successfully",
  "status": "success"
}

New character_id = 14


In [ ]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [78]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_path": "outputs/generated_audio_1748992638.wav",
  "character_name": "J. Robert Oppenheimer Local",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"chunk_id\": 6,\n  \"keywords\": \"project, weapons, nuclear, alamos laboratory, new\",\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"source\": \"oppenheimer_knowledge.txt\"\n}\n\nDocument 2:\nHe is often referred to as the \"father of the atomic bomb\" for his role as the scientific director of the Manhattan Project during World War II.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), World War II (EVENT)\",\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"chunk_id\": 2,\n  \"keywords\": \"role, referred, scientific, bomb role, world\"\n

In [41]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_path": "outputs/generated_audio_1748986540.wav",
  "character_name": "J. Robert Oppenheimer",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"keywords\": \"project, los alamos, project oppenheimer, weapons, mexico nuclear\",\n  \"chunk_id\": 6,\n  \"source\": \"oppenheimer_knowledge.txt\"\n}\n\nDocument 2:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"chunk_i

In [27]:
# Cell 4: List all characters in the database
chars_resp = get_characters()
print("Get-characters response:")
print(json.dumps(chars_resp, indent=2))


Get-characters response:
{
  "characters": [
    {
      "created_at": "Mon, 02 Jun 2025 23:49:33 GMT",
      "id": 5,
      "image_path": "storage/j._robert_oppenheimer/image_oppenheimer.png",
      "llm_model": "gpt-3.5-turbo",
      "name": "J. Robert Oppenheimer"
    }
  ],
  "status": "success"
}


In [4]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


KeyboardInterrupt: 

In [2]:
# Cell 5: Load the newly created character’s models into memory
character_id = 12
load_resp = load_character(character_id)
print(f"Load-character response for ID {character_id}:")
print(json.dumps(load_resp, indent=2))

# (Use the character_id obtained in Cell 3)
# if 'character_id' in locals():
#     load_resp = load_character(character_id)
#     print(f"Load-character response for ID {character_id}:")
#     print(json.dumps(load_resp, indent=2))
# else:
#     print("Error: character_id not defined. Run Cell 3 first.")


Load-character response for ID 12:
{
  "character_id": 12,
  "message": "Models loaded for character J. Robert Oppenheimer 7",
  "status": "success"
}


In [ ]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [ ]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [100]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_url": "http://localhost:5000/serve-audio/generated_audio/generated_audio_1749003837.wav",
  "character_name": "J. Robert Oppenheimer 7",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"chunk_id\": 6,\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"keywords\": \"los alamos, alamos laboratory, nuclear, mexico nuclear, weapons\"\n}\n\nDocument 2:\nHe is often referred to as the \"father of the atomic bomb\" for his role as the scientific director of the Manhattan Project during World War II.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), World War II (EVENT)\",\n  \"keywords\": \"father atomic, scientific, referred, world, director manhattan\

In [ ]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_path": "outputs/generated_audio_1748987406.wav",
  "character_name": "J. Robert Oppenheimer 2",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"chunk_id\": 6,\n  \"keywords\": \"mexico nuclear, weapons, project oppenheimer, nuclear, project\",\n  \"source\": \"oppenheimer_knowledge.txt\"\n}\n\nDocument 2:\nHe is often referred to as the \"father of the atomic bomb\" for his role as the scientific director of the Manhattan Project during World War II.\n\nMetadata:\n{\n  \"chunk_id\": 2,\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"keywords\": \"atomic bomb, bomb role, father atomic, role, referred\",\n  \"entities\": \"the Manhattan Project (ORG), World 

In [47]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


Ask-question-text response:
{
  "audio_path": "outputs/generated_audio_1748987827.wav",
  "character_name": "J. Robert Oppenheimer 2",
  "knowledge_context": "Document 1:\nDuring the Manhattan Project, Oppenheimer led the Los Alamos Laboratory in New Mexico, where the first nuclear weapons were designed and built.\n\nMetadata:\n{\n  \"keywords\": \"mexico nuclear, weapons, project oppenheimer, nuclear, project\",\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"entities\": \"the Manhattan Project (ORG), Oppenheimer (PERSON), the Los Alamos Laboratory (ORG), New Mexico (GPE), first (ORDINAL)\",\n  \"chunk_id\": 6\n}\n\nDocument 2:\nHe is often referred to as the \"father of the atomic bomb\" for his role as the scientific director of the Manhattan Project during World War II.\n\nMetadata:\n{\n  \"keywords\": \"atomic bomb, bomb role, father atomic, role, referred\",\n  \"source\": \"oppenheimer_knowledge.txt\",\n  \"chunk_id\": 2,\n  \"entities\": \"the Manhattan Project (ORG), World 

In [1]:
# Cell 6: Ask a text question to the character
# (Ensure character models are loaded from Cell 5)
if 'character_id' in locals():
    question = "What inspired you to lead the Manhattan Project?"
    text_resp = ask_question_text(character_id, question)
    print("Ask-question-text response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")


🔄 Testing styled text generation with OpenAI...
❌ OpenAI styled text generation failed: name 'get_style_data' is not defined
This might be due to missing API key or network issues


In [105]:
if 'character_id' in locals():
    audio_file_path = "./test_data/sample_question.wav"
    text_resp = ask_question_audio(character_id, audio_file_path)
    print("Ask-question-audio response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [ ]:
# Add this to your notebook or create a separate script
import sys
sys.path.append('./libraries')

from libraries.llm.preprocess import download_model

# Download the Gemma GGUF model
model_name = "https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf"
model_type = "huggingface"

try:
    local_path = download_model(model_name, model_type)
    print(f"✅ Model downloaded successfully to: {local_path}")
except Exception as e:
    print(f"❌ Download failed: {e}")

❌ Download failed: Failed to download model https://huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf: 404 Client Error. (Request ID: Root=1-683f7efa-6a7e56d1409dae297f8a8c35;1a21f988-1ff4-4ec9-864d-6b50765bdf81)

Repository Not Found for url: https://huggingface.co/https/resolve/main///huggingface.co/google/gemma-3-4b-it-qat-q4_0-gguf/blob/main/gemma-3-4b-it-q4_0.gguf.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication


In [115]:
if 'character_id' in locals():
    text_resp = ask_question_audio_live(character_id)
    print("Ask-question-audio-live response:")
    print(json.dumps(text_resp, indent=2))
else:
    print("Error: character_id not defined. Run Cells 3 and 5 first.")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))